# CS587:  Neural Networks & Learning of Hierarchical Representation
## Assignment 4a
### Vasileios Papageridis - csd4710

---

# CS - 587 : Exercise 4a ~ Transfer Learning
## Scope:
The goal of this assignment is to get familiar with fine-tunning in a new dataset a Convolutional Neural Network (CNN) that has been trained in another dataset, taking advantage of transfer learning.

In your assignment you will be fine-tunning **AlexNet**, a popular CNN architecture, that has been pretrained on the ImageNet dataset. Your network will be finetuned for the task of recognizing art painting categories in a large dataset of art painting images, known as Wikiart.

The WikiArt dataset, which consists of `3000 images of paintings` of arbitrary sizes `from 10 different styles` - Baroque, Realism, Expressionism, etc.

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
from Utilities.wikiart_dataset import WikiArtDataset

train_annotations = os.path.join('Utilities', 'data', 'train.txt')
test_annotations = os.path.join('Utilities', 'data', 'test.txt')

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_set = WikiArtDataset(train_annotations, preprocess)
test_set = WikiArtDataset(test_annotations, preprocess)
print(f"Training set: {len(train_set)} images")
print(f"Test set: {len(test_set)} images")

Training set: 3177 images
Test set: 784 images


In [3]:
# Learning params
learning_rate = 0.01
num_epochs = 5
batch_size = 128

# Network params
num_classes = 10

# Pretrained Model
For all of our image generation experiments, we will start with a convolutional neural network which was pretrained to perform image classification on ImageNet. We can use any model here, but for the purposes of this assignment we will use AlexNet

In [4]:
# load the alexnet model pretrained on ImageNet
alexnet = models.alexnet(weights='IMAGENET1K_V1')

# freeze the feature parameters
for param in alexnet.parameters():
    param.requires_grad = False

print(alexnet)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\billi/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:43<00:00, 5.61MB/s] 

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [5]:
##############################################################################################
# TODO: Modify the model architecture:                                                       #
# 1. Replace the last Fully-Connected (linear) layer with a new linear layer                 #
# 2. Replace the last 2 Fully-Connected layers with a new linear layer                       #
# Hint: You can access the layers of the model using alexnet.features and alexnet.classifier #
# Hint: You can access multiple layers of the classifier using alexnet.classifier.children() #
# Hint: You can remove layers using list slicing with the appropriate indices                #
# Hint: You can add layers using add_module()                                                #
##############################################################################################

num_layers_to_remove = 1
if num_layers_to_remove == 1:
    #############################################################
    # TODO: remove the last linear layer and add new classifier #
    #############################################################
    num_features = alexnet.classifier[6].in_features
    alexnet.classifier[6] = nn.Linear(num_features, num_classes)
elif num_layers_to_remove == 2:
    ##################################################################
    # TODO: remove the last 2 linear layers and add a new classifier #
    ##################################################################
    num_features = alexnet.classifier[4].in_features

    new_classifier = nn.Sequential(
        *alexnet.classifier[:-2],
        nn.Linear(num_features, num_classes)
    )

    alexnet.classifier = new_classifier

alexnet = alexnet.to(device)
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [10]:
##################################################################
# TODO: Implement the following:                                 #
# (a) the train and test data loaders                            #
# (b) losss function (Soft-max Cross Entropy)                    #
# (c) the optimization process using Stochastic Gradient Descent #
# Create summaries in tensorboard for:                           #
#  - the loss                                                    #
#  - the accuracy                                                #
##################################################################
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=learning_rate)

def accuracy(model, data_loader):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [11]:
########################################
# TODO: use ΤensorΒoard to visualize   #
# the computational graph of the model #
########################################
writer = SummaryWriter('runs/AlexNet_FineTuning')

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        ###########################################################################
        # TODO: backpropagation process (forward, loss, backward, update weights) #
        ###########################################################################
        pred = alexnet(images)
        loss_val = loss(pred, labels)

        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()
        ##########################################################
        # TODO: use TensorBoard to visualize for each iteration: #
        #        - the training loss                             #
        ##########################################################
        writer.add_scalar('training loss', loss_val.item(), epoch * len(train_loader) + i)

    ###########################################################
    # TODO: calculate the accuracy for the train and test set #
    ###########################################################
    acc_train = accuracy(alexnet, train_loader)
    acc_test = accuracy(alexnet, test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, train accuracy: {acc_train:.3f} test accuracy: {acc_test:.3f}")
    ######################################################
    # TODO: use TensorBoard to visualize for each epoch: #
    #       the train & test accuracy                    #
    ######################################################
    writer.add_scalar('Accuracy/Train', acc_train, epoch)
    writer.add_scalar('Accuracy/Test', acc_test, epoch)

Epoch 1/5, train accuracy: 0.448 test accuracy: 0.346
Epoch 2/5, train accuracy: 0.519 test accuracy: 0.389
Epoch 3/5, train accuracy: 0.560 test accuracy: 0.413
Epoch 4/5, train accuracy: 0.589 test accuracy: 0.388
Epoch 5/5, train accuracy: 0.658 test accuracy: 0.409
